In [1]:
from pprint import pprint
import jmespath
import requests
from bs4 import BeautifulSoup as BS4
from textwrap import dedent
import re

In [2]:
req = requests.get('https://www.selver.ee/')

In [3]:
soup = BS4(req.content, features='html.parser')

In [4]:
#esilehelt URL otsingu regex
url_class = re.compile(r'(item level0 nav.\w+.+\d)')
#Hinna ja toote otsingu regex
product_class = re.compile(r'(col-lg-3 \w+).+') 
product_price = re.compile(r'("price")')

In [5]:
#Leia kõik HTML Li tagilt mis hoiab URLi
find_tag = soup.find_all('li',{'class':url_class})

In [6]:
#leia URL'id ja lista toote nimekirja pikkus
get_urls = [i.find_next('a').get('href')+"/?limit=96&p=" for i in find_tag]

In [7]:
get_urls

['https://www.selver.ee/minu-tooted/?limit=96&p=',
 'https://www.selver.ee/uudistooted/?limit=96&p=',
 'https://www.selver.ee/soodushinnaga-tooted/?limit=96&p=',
 'https://www.selver.ee/eritooted/?limit=96&p=',
 'https://www.selver.ee/hulgipakkumised/?limit=96&p=',
 'https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=',
 'https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=',
 'https://www.selver.ee/piimatooted-munad-void/?limit=96&p=',
 'https://www.selver.ee/juustud/?limit=96&p=',
 'https://www.selver.ee/leivad-saiad-kondiitritooted/?limit=96&p=',
 'https://www.selver.ee/valmistoidud/?limit=96&p=',
 'https://www.selver.ee/kuivained-hoidised/?limit=96&p=',
 'https://www.selver.ee/maitseained-ja-puljongid/?limit=96&p=',
 'https://www.selver.ee/kastmed-olid/?limit=96&p=',
 'https://www.selver.ee/maiustused-kupsised-naksid/?limit=96&p=',
 'https://www.selver.ee/kulmutatud-toidukaubad/?limit=96&p=',
 'https://www.selver.ee/joogid/?limit=96&p=',
 'https://www.selver.ee/lastekaubad/?limi

In [11]:
#Kogu html linkide kogust
product_html = requests.get(get_urls[5])
#puhasta ilusa seebiga
soup = BS4(product_html.content, features='html.parser')
#leia toote ja hinna kategoorijad li tagist
find_product_group = soup.find_all('li',{'class':product_class})
#leia hinnad
find_product_group_prices = soup.find_all('span',{'class':'price'})
#võta linki pealkirjad ehk toote nimed
find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
#leia kõik hinnad
prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]

In [8]:
html_list = list()
dict_of_products = dict()

In [339]:
#start of the urls and end of the URLS
for z in range(5,23):
    #Find product last page
    product_html = requests.get(get_urls[z])
    #puhasta ilusa seebiga
    soup = BS4(product_html.content, features='html.parser')
    try:
        find_product_last_page = soup.find('a',{'class':'last'}).text
    except AttributeError:
        find_product_last_page = 3
    
    for i in range(1,int(find_product_last_page)+1):
        product_html = requests.get(get_urls[z]+str(i))
        soup = BS4(product_html.content, features='html.parser')
        html_list.append(soup)  
        print(f"Page URL: {get_urls[z]+str(i)} Last page is: {find_product_last_page}")

    for y in range(len(html_list)):  
        find_product_group = html_list[y].find_all('li',{'class':product_class})
        find_product_group_prices = html_list[y].find_all('span',{'class':'price'})
        find_product_group_titles = [i.find_next('a').get('title') for i in find_product_group]
        prices = [dedent(i.text.replace(u'\xa0',' ')) for i in find_product_group_prices if len(i) > 1]
        for t,f in zip(find_product_group_titles,prices):
            dict_of_products.update({t:re.split(r'\s+([0-9]+.[0-9]+\s+./[A-z]+)',f)[:2]})

Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=1 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=2 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=3 Last page is: 4
Page URL: https://www.selver.ee/puu-ja-koogiviljad/?limit=96&p=4 Last page is: 4
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=1 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=2 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=3 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=4 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=5 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=6 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=7 Last page is: 9
Page URL: https://www.selver.ee/liha-ja-kalatooted/?limit=96&p=8 Last page is: 9
Page URL: https://www.selver

Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=10 Last page is: 12
Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=11 Last page is: 12
Page URL: https://www.selver.ee/enesehooldustarbed/?limit=96&p=12 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=1 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=2 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=3 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=4 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=5 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=6 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=7 Last page is: 12
Page URL: https://www.selver.ee/majapidamis-ja-kodukaubad/?limit=96&p=8 Last page is: 12
Page URL: https://www.selver.ee/majapid

In [36]:
import pandas as pd

In [395]:
df = pd.DataFrame(index=dict_of_products.keys(),data=[y for x,y in dict_of_products.items()], columns=['Hind','Hind/Kogus)'])

In [356]:
from sqlalchemy import create_engine

In [357]:
table = create_engine('sqlite:///selver_data.db', echo=True)
table_c = table.connect()

2021-03-02 21:06:56,552 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-03-02 21:06:56,556 INFO sqlalchemy.engine.base.Engine ()
2021-03-02 21:06:56,564 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-03-02 21:06:56,566 INFO sqlalchemy.engine.base.Engine ()


In [358]:
df.to_sql('Selver Items and Prices', table_c, if_exists='append')

2021-03-02 21:07:56,583 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Selver Items and Prices")
2021-03-02 21:07:56,585 INFO sqlalchemy.engine.base.Engine ()
2021-03-02 21:07:56,589 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Selver Items and Prices")
2021-03-02 21:07:56,593 INFO sqlalchemy.engine.base.Engine ()
2021-03-02 21:07:56,603 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Selver Items and Prices" (
	"index" TEXT, 
	"Hind" TEXT, 
	"Hind/Kogus)" TEXT
)


2021-03-02 21:07:56,605 INFO sqlalchemy.engine.base.Engine ()
2021-03-02 21:07:56,616 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-02 21:07:56,618 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_Selver Items and Prices_index" ON "Selver Items and Prices" ("index")
2021-03-02 21:07:56,620 INFO sqlalchemy.engine.base.Engine ()
2021-03-02 21:07:56,634 INFO sqlalchemy.engine.base.Engine COMMIT
2021-03-02 21:07:56,669 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-03-02 21:07:56,737 

In [399]:
df.reset_index(inplace=True)

In [1]:
df

NameError: name 'df' is not defined

In [391]:
df

,Toode,hind,Hind/Kogus,"(hind, Hind/Kogus)"
0,"Banaan Chiquita, kg",NaN,NaN,"[1,39 €, 1,39 €/kg]"
1,"Kurk Luunja, kg",NaN,NaN,"[5,99 €, 5,99 €/kg]"
2,"Kirssploomtomat, 250 g",NaN,NaN,"[1,29 €, 5,16 €/kg]"
3,"Kirsskobartomat, 500 g",NaN,NaN,"[2,99 €, 5,98 €/kg]"
4,"Apelsin Navel, kg",NaN,NaN,"[1,59 €, 1,59 €/kg]"
...,...,...,...,...
10350,"Šokolaaditort, EESTI PAGAR, 1 kg",NaN,NaN,"[12,49 €, 12,49 €/kg]"
10351,"Kakaokeeks Kinder Delice, FERRERO, 42g",NaN,NaN,"[0,59 €, 14,05 €/kg]"
10352,"Sarapuupähklitega piimašokolaadipallid Lindor,...",NaN,NaN,"[6,99 €, 34,95 €/kg]"
10353,"Jahvatatud kohv Dansk Guld purgis, MERRILD, 250g",NaN,NaN,"[4,99 €, 19,96 €/kg]"
